In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('../')

# Propositional Classifiers:

In this notebook I try to design classifiers that act on the set of all blocks and output a tiebreaker binary label having access only to the features given by Henry in the focal students dataset. Those features are:

1. Total number of students per block (used for normalization purposes)
2. Number of FRL students per block
3. Number of AALPI students per block
4. Number of FRL and AALPI students per block (i.e. intersection of those)

The classifiers will evaluate a logical proposition with those features. For example, an "AND" classifier can be of the form:

$$ \text{AALPI} \geq 0.5 \quad \text{and} \quad \text{FRL} \geq 0.7 $$

This classifier will give an equity tiebreaker to a block if and only if that block has over 50% of its students in the AALPI racial group and over 70% of its students receiving FRL.

Currently we do not have a systematic way to think of these types of propositions. But we can evaluate their performance based on false positive and false negative rates. In the case of two parameters (i.e. two numeric comparisons), it is possible to visualize the precision-recall curve.

## 1. Preparing the Data

In [ ]:
from src.d01_data.block_data_api import BlockDataApi
from src.d02_intermediate.block_data_cleaning import frl_raw_to_pc

To use the propositional classifiers, we need to format the FRL data. The block_data_cleaning script has the function to turn raw frl data into the input we need:

In [ ]:
block_data_api = BlockDataApi()
frl_df = block_data_api.get_data(frl=True, user="gabriel")
frl_df_pc = frl_raw_to_pc(frl_df)

In [ ]:
frl_df_pc

The above dataframe contains all the possible columns we could use in our classification. Most of the times we will deal with relative comparisons (is over half of a block focal?), but the absolute counts are important to give an idea of false positives and false negatives. Besides, we might want to mix absolute and relative counts: blocks with a great number of students may require that we look at absolute counts, otherwise we could be neglecting too many focal students.

## 2. Class Syntax

We load the propositional classifier classes from the modelling library:

In [ ]:
from src.d04_modeling import propositional_classifier as pc

In its most general form, the propositional classifier class takes as an input a list of features that we will be using for evaluation, a list of logical operators ("and" or "or"), and a list of comparisors ($\geq$, $\leq$, =). By default, this comparisor lists is a sequence of $\geq$ since that is the most likely case. The lists must be in the order of the statement we want to construct, and notice that there will always be one less operator than features. For example:

In [ ]:
pc1 = pc.Propositional_Classifier(["AALPI (%)", "FRL (%)", "BOTH"], ["and", "or"])
pc1.statement

Note that the parameters are not required upon initialization. Rather, the statement is constructed so we can input parameters when doing the predictions. This way we can vary parameters and build precision-recall cruves.

Simple and/or classifiers have their own child class, in which we only need to pass the features (and comparisors if not default):

In [ ]:
pc2 = pc.AND_Classifier(["AALPI (%)", "FRL (%)"])
pc2.statement

In [ ]:
pc3 = pc.OR_Classifier(["AALPI (%)", "FRL (%)"])
pc3.statement

Some logical statements need parentheses. Some do not, but we would rather read them with parentheses as that is easier (for example, the first example pc1 is hard to interpret without parenthesis---computer evaluates it in order). Simply pass a tuple of features as an element. Note that operands must still be of the correct length!

In [ ]:
pc4 = pc.Propositional_Classifier(["AALPI (%)", ("FRL (%)", "BOTH (%)"), "BOTH"], ["or", "and", "or"])
pc4.statement

Once we have initialized our statement we can use the predict method with the appropriate data and parameters to do a round of prediction.

In [ ]:
params1 = [0.5, 0.8, 10] #parameters must match the features passed, in the order. Note scales (relative vs. absolute)
pred1 = pc1.predict(frl_df_pc, params1)
pred1

This binary array tells us which blocks receive the tiebreaker. We can visualize the result in the San Francisco map:

In [ ]:
pc1.display_map(frl_df_pc, pred1)

The confusion matrix of this classifier are saved on a dictionary whose key is the parameter tuples for which a predicition has been made:

In [ ]:
pc1.confusion_dict

Using that we can retrive any rate for evaluating purposes. We can get the FPR and FNR for example:

In [ ]:
fpr, fnr = pc1.get_rates(params1, which=["fpr", "fnr"])
print("False positive rate is {0:.2f} %".format(100*fpr))
print("False negative rate is {0:.2f} %".format(100*fnr))

To interpret the above map: we wanted the FRL percentage to be very high (above 80%) and the AALPI percentage to be at least half; OR if there were at least 10 students in a block in the intersection count, the block would be given a tiebreaker regardless of its relative composition. We can see in the map that due to the AALPI criterion having to be satisfied most blocks that received a tiebreaker are in the SouthEast (where racial minorities are more concentrated). This criterion is very restrictive: the false negative rate is super high, meaning that we "missed" a lot of focal students. However, very few non-focal students received an advantage (less that 10%).

## 3. Exploring the parameter space

Ideally we would like to explore several points for the trade-off between FP and FN.

In one-dimensional parameter spaces (i.e. only one feature is pased to the classifier, so that we have only one parameter) this can be done via analysis of the ROC curve (similar to precision-recall):

In [ ]:
pc5 = pc.Propositional_Classifier(["BOTH (%)"], [])

In [ ]:
pc5.statement

In [ ]:
pc5.display_ROCcurve(frl_df_pc, params_range=(0,1), n_params=100)

In two-dimensional parameter spaces (i.e. only two features are pased to the classifier, so that we have only two parameters) this can be done via analysis of two matrices of false positives and false negatives. This would be equivalent to a ROC surface.

In [ ]:
pc2.statement

In [ ]:
pc2.display_heat(frl_df_pc, params_range=(0,1), n_params=10)

In [ ]:
pc3.statement

In [ ]:
pc3.display_heat(frl_df_pc, params_range=(0,1), n_params=10)